In [ ]:
import re
import json
import ast
import openai
import requests
import urllib.parse 
from selenium import webdriver

def convert_to_smiles(iupac_name):
    # URL encode the chemical name
    encoded_name = urllib.parse.quote(iupac_name)

    # the base URL of the web service
    base_url = "https://opsin.ch.cam.ac.uk/opsin/"

    # the URL of the request
    url = base_url + encoded_name + ".smi"

    # send the GET request
    response = requests.get(url)

    # return the SMILES code
    return response.text


def smiles_to_zinc_name(smiles):
    # Validate that smiles is a string containing valid SMILES characters
    pattern = r'^[a-zA-Z0-9()=+#:@/.\\%-]*$'
    if not isinstance(smiles, str) or not re.match(pattern, smiles):
        print("The input must be a valid SMILES string containing only valid characters.")
        return "N/A"
    
    # Setting up Chrome options for headless browsing
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)

    # Navigating to the URL
    url = "https://zinc20.docking.org/substances/home/"
    driver.get(url)

    # Finding the search box and entering the smiles string
    search_box = driver.find_element("name", "q")
    search_box.send_keys(smiles)

    # Finding and clicking the search button
    search_button = driver.find_element("xpath", "//button[text()='Search']")
    search_button.click()

    # Extracting image tags using regular expressions
    pattern_img = r'<img\s+src=".*?"\s+alt=".*?">'
    img_tags = re.findall(pattern_img, driver.page_source)

    # Quitting the driver
    driver.quit()

    # If img_tags is not empty, extract the alt text, else return "N/A"
    if img_tags:
        pattern_alt = r'alt="(.*?)"'
        alt_texts = re.findall(pattern_alt, img_tags[0])
        return alt_texts[0]
    else:
        return "N/A"



def get_inventory(zinc_id):
    # If zinc_id is "N/A", return unavailable
    if zinc_id == "N/A":
        return "Availability: N/A  Vendors: N/A"
    else:
        # Constructing the URL using the medical name
        url = "https://zinc20.docking.org/substances/" + zinc_id
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=options)

        # Navigating to the URL
        driver.get(url)

        # Extracting availability information
        availability = driver.find_element("xpath", "/html/body/div[1]/div/div/div[1]/div[1]/div[3]/table[1]/tbody/tr/td[2]").accessible_name
        
        # Extracting vendors number information
        vendors_number = driver.find_element("xpath", "/html/body/div/div/div/div[3]/div/div[1]/div[1]/h3").accessible_name
        num = re.findall(r'\d+', vendors_number)[0]

        # Extracting annotated catalogs information
        cate_num = driver.find_element("xpath", "/html/body/div/div/div/div[3]/div/div[2]/div[1]/h3/a").accessible_name
        cnum = re.findall(r'\d+', cate_num)[0]

        # Quitting the driver
        driver.quit()

        return "Availability: " + availability + "  Vendors: " + num + "  Annotated Catalogs: " + cnum

def check_inventory(smiles):
    zinc_id = smiles_to_zinc_name(smiles)
    return get_inventory(zinc_id)
    

def execute_function_call(function_name, arguments):
    try:
        # Check if the function exists and call it
        if function_name in globals():
            return globals()[function_name](**arguments)
        else:
            # Function does not exist
            return f"Error: function {function_name} does not exist"
    except Exception as e:
        # Handle any errors that occur during function execution
        return f"Function execution failed with error: {e}"
    
    

class ChatBotWithFunctions:
    def __init__(self, api_key, model="gpt-4-0613", helper_functions=None):
        # Set the API key and model
        openai.api_key = api_key
        self.model = model
        self.system_prompt = """You are a chemistry expert. 
        If the user provide you with a common name of the chemical instead of standard IUPAC name, you shouold first convert
        it to IUPAC name based on your knowledge.
        
        """
        
        #back-up prompt
        """You are a chemistry expert. Do not assume if you do not know the exact parameter in the function. 
        If the input parameter is empty, please ask the user to provide.
        If the user provide you with a common name of the chemical instead of standard IUPAC name, you shouold first convert
        it to IUPAC name based on your knowledge, and then use it as the input paramter for the function calling.
   
        """
        
        

        # Store the helper functions
        self.helper_functions = helper_functions or []
        
        # Initialize the chat history
        self.chat_history = [{"role": "system", "content":self.system_prompt }]

    def chat(self, user_input, temp=None, max_tokens=None, max_fc=20):
        # Add the user's input to the chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Generate a chat completion with the current chat history and helper functions
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=self.chat_history,
            functions=self.helper_functions,
            temperature=temp,
            max_tokens=max_tokens
        )

        # Get the assistant's message from the response
        assistant_message = response['choices'][0]['message']
        
        #limit the number of function call
        number_fc = 0
        max_function_call = max_fc
        
        # If the assistant's message contains a function call
        while assistant_message.get("function_call") and (number_fc < max_function_call) :
            # Parse the function call data
            #data = json.loads(assistant_message["function_call"])
            #function_name = data["name"]
            #arguments = ast.literal_eval(data["arguments"])

            data = assistant_message["function_call"]
            function_name = data["name"]
            try:
                arguments = ast.literal_eval(data["arguments"])
            except SyntaxError:
                # Handle the error here, perhaps by logging it or providing a default value
                if isinstance(data["arguments"], str):
                    try:
                        arguments = json.loads(data["arguments"])
                        
                        parsed_data = json.loads(data["arguments"])
                        if "smiles" in parsed_data:
                            arguments = parsed_data["smiles"]
                        elif "iupac_name" in parsed_data:
                            arguments = parsed_data["iupac_name"]
                        else:
                            raise ValueError("Neither 'smiles' nor 'iupac_name' found in arguments.")
        
                    except (json.JSONDecodeError, ValueError):
                        arguments = {}
                        print("Failed to parse arguments from below message, setting to empty")
                        print(assistant_message)
                else:
                    arguments = {}
                    print("Arguments are not in a recognizable format, setting to empty")


            # If a function with the given name exists in the global scope
            if function_name in globals():
                print(f"Function:'{function_name}' was used with input '{arguments}'.")
                # Call the function with the provided arguments and store the result
                function_results = execute_function_call(function_name, arguments)
                print(f"The result is:'{function_results}' ")
                self.chat_history.append({"role": "function", "name": function_name, "content": function_results})

                # Generate a new chat completion with the updated chat history
                response = openai.ChatCompletion.create(
                    model=self.model,
                    messages=self.chat_history,
                    functions=self.helper_functions
                )

                # Update the assistant's message with the new response
                #print(response)
                assistant_message = response['choices'][0]['message']
            else:
                print(f"No function named '{function_name}' was found.")
            
            #count number of total function calls, should not be larger than the max function call number
            number_fc+=1
            
        # Get the content of the assistant's message
        assistant_message_content = assistant_message['content']

        # Add the assistant's message to the chat history
        self.chat_history.append({"role": "assistant", "content": assistant_message_content})

        # Return the assistant's message
        return assistant_message_content

    def reset(self):
        # Reset the chat history
        self.chat_history = [{"role": "system", "content": self.system_prompt}]

functions_definition = [
    {
        "name": "convert_to_smiles",
        "description": "Converts an IUPAC name to a SMILES code",
        "parameters": {
            "type": "object",
            "properties": {
                "iupac_name": {
                    "type": "string",
                    "description": "The IUPAC name of the chemical"
                }
            },
            "required": ["iupac_name"]
        }
    },
    {
        "name": "check_inventory",
        "description": "Checks the inventory for a given SMILES code. Output is the commericial availability and number of vendors for this compound.",
        "parameters": {
            "type": "object",
            "properties": {
                "smiles": {
                    "type": "string",
                    "description": "The SMILES code of the chemical"
                }
            },
            "required": ["smiles"]
        }
    }
]




In [ ]:
# Test the class
bot = ChatBotWithFunctions('Put Your Real OpenAI API Here', helper_functions=functions_definition)



print(bot.chat("Can I buy MOF linker H3BTC?"))
print('\n')
bot.reset() 